In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using OrdinaryDiffEq
using ForwardDiff
using Plots

In [ ]:
#Pendulum Parameters
ℓ = 1.0
m = 1.0
g = 9.81

In [ ]:
#Reference Solution w/ODE Solver

#Classical pendulum dynamics
function f(x,p,t)
    q = x[1]
    v = x[2]
    
    v̇ = -(g/ℓ)*sin(q)
    
    ẋ = [v; v̇]
end

#initial conditions
x0 = [pi/2; 0]

#Simulate
tspan = (0.0,100.0)
prob = ODEProblem(f,x0,tspan)
sol = solve(prob,Tsit5());
#sol = solve(prob,Tsit5(),abstol=1e-5,reltol=1e-5);
#sol = solve(prob,Tsit5(),abstol=1e-6,reltol=1e-6);

In [ ]:
plot(sol,idxs=(0,1))

In [ ]:
#Energy Functions

function T(θ̇)
    0.5*m*ℓ*ℓ*θ̇*θ̇
end

function U(θ)
    m*g*ℓ*(1-cos(θ))
end

function H(x)
    U(x[1]) + T(x[2])
end

In [ ]:
#Plot total energy
h = 0.01 #time step
tk = 0:h:tspan[2]

N = length(tk)
E = zeros(N)
for k = 1:N
    E[k] = H(sol(tk[k]))
end

plot(tk,E)

In [ ]:
h = 0.001 #time step
tk = 0:h:100

In [ ]:
#Define discrete Lagrangian
function L(q,q̇)
    T(q̇) - U(q)
end

function Ld(q1,q2)
    θm = 0.5*(q1+q2)
    θ̇m = (q2-q1)/h
    return h*L(θm,θ̇m)
end

In [ ]:
#Let's do this again with the discrete Legendre transform

function D1Ld(q1,q2)
    ForwardDiff.derivative(dq1->Ld(dq1,q2),q1)
end

function D2Ld(q1,q2)
    ForwardDiff.derivative(dq2->Ld(q1,dq2),q2)
end

#Initial conditions
q1 = x0[1]
v1 = x0[2]
p1 = m*v1

qhist = zeros(length(tk))
qhist[1] = q1
phist = zeros(length(tk))
phist[1] = p1

for k = 1:(length(tk)-1)
    qhist[k+1] = qhist[k]
    
    #Calculate residual with right discrete Legendre transform
    r = phist[k] + D1Ld(qhist[k],qhist[k+1])
    
    #Newton's method to solve for q_k+1
    while norm(r) > 1e-12
        R = ForwardDiff.derivative(dqn->D1Ld(qhist[k],dqn),qhist[k+1])
        qhist[k+1] = qhist[k+1] - R\r
        r = phist[k] + D1Ld(qhist[k],qhist[k+1])
    end
    
    #Update momentum (left discrete Legendre transform)
    phist[k+1] = D2Ld(qhist[k],qhist[k+1])
end

In [ ]:
plot(qhist)

In [ ]:
#Plot energy

E2 = zeros(length(tk))
for k = 1:(length(tk))
    E2[k] = (0.5/m)*phist[k]*phist[k] + U(qhist[k])
end

plot(E2)
#plot!(E)

In [ ]:
#Implicit Midpoint

xhist = zeros(2,length(tk))
xhist[:,1] .= x0

for k = 1:(length(tk)-1)
    xhist[:,k+1] .= xhist[:,k]
    r = xhist[:,k+1] - xhist[:,k] - h*f(0.5*(xhist[:,k+1]+xhist[:,k]),[],[])
    
    #Newton's method
    while norm(r) > 1e-12
        R = I - 0.5*h*ForwardDiff.jacobian(dx->f(dx,[],[]),0.5*(xhist[:,k+1]+xhist[:,k]))
        xhist[:,k+1] .= xhist[:,k+1] - R\r
        r = xhist[:,k+1] - xhist[:,k] - h*f(0.5*(xhist[:,k+1]+xhist[:,k]),[],[])
    end
end

In [ ]:
plot(tk,xhist[1,:])
plot!(tk,qhist)

In [ ]:
#Plot total energy

E3 = zeros(length(tk))
for k = 1:(length(tk))
    E3[k] = H(xhist[:,k])
end

In [ ]:
plot(E2)
plot!(E3)

In [ ]:
qhist-xhist[1,:]

In [ ]:
phist-xhist[2,:]